In [2]:
#ds=xr.Dataset({'qc':qcXR,'qr':qrXR,'qi':qiXR,'qs':qsXR,'qg':qgXR,'qns':qnsXR,'qng':qngXR,'qnr':qnrXR,'qni':qniXR,'qv':qvXR,     'press':pressXR,'temp':tempXR,'dz':dzXR,'qc_tend':qc_tendXR,'qr_tend':qr_tendXR,'qi_tend':qi_tendXR,'qs_tend':qs_tendXR,'qg_tend':qg_tendXR,'qns_tend':qns_tendXR,'qng_tend':qng_tendXR,'qnr_tend':qnr_tendXR,'qni_tend':qni_tendXR,'qv_tend':qv_tendXR,'temp_tend':temp_tendXR})

from netCDF4 import Dataset
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

with nc.Dataset("trainingData.nc") as fh:
    qc=fh.variables['qc'][:]
    qr=fh.variables['qr'][:]
    qi=fh.variables['qi'][:]
    qs=fh.variables['qs'][:]
    qg=fh.variables['qg'][:]
    qns=fh.variables['qns'][:]
    qng=fh.variables['qng'][:]
    qnr=fh.variables['qnr'][:]
    qni=fh.variables['qni'][:]
    qv=fh.variables['qv'][:]
    press=fh.variables['press'][:]
    temp=fh.variables['temp'][:]
    dz=fh.variables['dz'][:]
    qc_tend=fh.variables['qc_tend'][:]
    qr_tend=fh.variables['qr_tend'][:]
    qi_tend=fh.variables['qi_tend'][:]
    qs_tend=fh.variables['qs_tend'][:]
    qg_tend=fh.variables['qg_tend'][:]
    qns_tend=fh.variables['qns_tend'][:]
    qng_tend=fh.variables['qng_tend'][:]
    qnr_tend=fh.variables['qnr_tend'][:]
    qni_tend=fh.variables['qni_tend'][:]
    qv_tend=fh.variables['qv_tend'][:]
    temp_tend=fh.variables['temp_tend'][:]
    nt,nz=qc.shape
    

In [3]:
#scale variables using the standard scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

inputVars=["qc","qr","qi","qs","qg","qns","qng","qnr","qni","qv","press","temp","dz"]
outputVars=["qc_tend","qr_tend","qi_tend","qs_tend","qg_tend","qns_tend","qng_tend","qnr_tend","qni_tend","qv_tend","temp_tend"]
inputData={"qc":qc,"qr":qr,"qi":qi,"qs":qs,"qg":qg,"qns":qns,"qng":qng,"qnr":qnr,"qni":qni,"qv":qv,"press":press,"temp":temp,"dz":dz}
outputData={"qc_tend":qc_tend,"qr_tend":qr_tend,"qi_tend":qi_tend,"qs_tend":qs_tend,"qg_tend":qg_tend,"qns_tend":qns_tend,"qng_tend":qng_tend,"qnr_tend":qnr_tend,"qni_tend":qni_tend,"qv_tend":qv_tend,"temp_tend":temp_tend}
inputDataScaled={}
outputDataScaled={}
inputScalers={}
outputScalers={}
for var in inputVars:
    inputScalers[var]=StandardScaler()
    inputDataScaled[var]=inputScalers[var].fit_transform(inputData[var])
for var in outputVars:
    outputScalers[var]=StandardScaler()
    outputDataScaled[var]=outputScalers[var].fit_transform(outputData[var])

In [27]:
inputData=np.array([inputDataScaled[var] for var in inputVars]).T
outputData=np.array([outputDataScaled[var] for var in outputVars]).T
inputData=np.swapaxes(inputData,0,1)
outputData=np.swapaxes(outputData,0,1)
print(inputData.shape,outputData.shape)


(684284, 72, 13) (684284, 72, 11)


In [14]:
print(dz[0,0:64].sum())

15450.283


In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(inputData,outputData,test_size=0.4,random_state=42)

In [29]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(410570, 72, 13) (273714, 72, 13) (410570, 72, 11) (273714, 72, 11)


In [6]:
import pickle
pickle.dump(inputScalers,open("inputScalers.pkl","wb"))
pickle.dump(outputScalers,open("outputScalers.pkl","wb"))

In [35]:
from uNet1D import *

model=unet1D(input_size=(72,13), nout=11, n_filters=[16])
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 72, 13)]             0         []                            
                                                                                                  
 conv1d_69 (Conv1D)          (None, 72, 16)               640       ['input_7[0][0]']             
                                                                                                  
 conv1d_70 (Conv1D)          (None, 72, 16)               784       ['conv1d_69[0][0]']           
                                                                                                  
 batch_normalization_30 (Ba  (None, 72, 16)               64        ['conv1d_70[0][0]']           
 tchNormalization)                                                                          

In [36]:
model.compile(optimizer='adam',loss='mse',metrics=['mse'])
hist=model.fit(X_train,y_train,epochs=30,batch_size=32,validation_data=(X_test,y_test))
#y_=model.predict(X_train[0:1,:,:])
#print(y_.shape)

Epoch 1/30
12831/12831 [==============================] - 295s 23ms/step - loss: 0.4944 - mse: 0.4944 - val_loss: 0.3699 - val_mse: 0.3699
Epoch 2/30
12831/12831 [==============================] - 293s 23ms/step - loss: 0.3934 - mse: 0.3934 - val_loss: 0.3338 - val_mse: 0.3338
Epoch 3/30
12831/12831 [==============================] - 293s 23ms/step - loss: 0.3836 - mse: 0.3836 - val_loss: 0.3210 - val_mse: 0.3210
Epoch 4/30
12831/12831 [==============================] - 293s 23ms/step - loss: 0.3657 - mse: 0.3657 - val_loss: 0.3132 - val_mse: 0.3132
Epoch 5/30
12831/12831 [==============================] - 293s 23ms/step - loss: 0.3614 - mse: 0.3614 - val_loss: 0.3151 - val_mse: 0.3151
Epoch 6/30
12831/12831 [==============================] - 294s 23ms/step - loss: 0.3475 - mse: 0.3475 - val_loss: 0.3173 - val_mse: 0.3173
Epoch 7/30
12831/12831 [==============================] - 294s 23ms/step - loss: 0.3379 - mse: 0.3379 - val_loss: 0.2980 - val_mse: 0.2980
Epoch 8/30
12831/12831 [===

KeyboardInterrupt: 

In [37]:
model.save("unet1D_MP_ML.keras")

In [ ]:
#load model
from keras.models import load_model
model=load_model("unet1D_MP_ML.keras")

In [38]:
import time
t0=time.time()
y_=model.predict(X_test[:,:,:])
t1=time.time()
print("time to predict=",t1-t0)

8554/8554 [==============================] - 46s 5ms/step
time to predict= 48.00339674949646


In [45]:
print(y_.shape[0]/(189*202))
for i,var in enumerate(outputVars):
    print(var,np.corrcoef(y_[:,:,i].flatten(),y_test[:,:,i].flatten())[0,1])

7.16941694169417
qc_tend 0.8670579489733449
qr_tend 0.7870705929611977
qi_tend 0.7859893413564109
qs_tend 0.8684453714733126
qg_tend 0.9094923433196229
qns_tend 0.5142148886485024
qng_tend 0.6888834950147088
qnr_tend 0.9133099971930863
qni_tend 0.5552458311705498
qv_tend 0.838947535990423
temp_tend 0.8399929499070017
